# MCM2009: Analytical solution Eqn. 6 and temperature profile Eqn. 5

In [ ]:
import numpy as np
from numpy import sqrt, pi, exp
from scipy.special import erf, erfc

In [ ]:
import matplotlib.pyplot as plt

From the article by Muhieddine, Canot, March (MCM2009), we have the following (Equations 5 and 6 in that paper).

$$x^{*} = \frac{x}{2\sqrt{\mu_l t}},
\mu = \frac{k}{\rho C}$$

## Eqn. 5:

$$T_l = T_w + (T_f - T_w) \frac{\textrm{erf}(x^{*})}{\textrm{erf}(x^{*}_{sl})}, 0 < x^{*} < x^{*}_{sl}$$

$$T_s = T_0 + (T_f - T_0)
\frac{\textrm{erfc}(\sqrt{\mu_l / \mu_s} x^{*})}
{\textrm{erfc}(\sqrt{\mu_l / \mu_s} x^{*}_{sl})}, x^{*}_{sl} < x^{*} < \infty
$$


$x^{*}_{sl}$ can be found using 

## Eqn. 6:

$$\frac{T_f - T_0}{T_f - T_w}
\frac{k_s}{k_l}
\sqrt{\frac{\mu_l}{\mu_s}}
\frac{\exp(-(\mu_l/\mu_s)(x^{*}_{sl})^2)}
{\textrm{erfc}(\sqrt{\mu_l/\mu_s}x^{*}_{sl})}
+ \frac{\exp(-(x^{*}_{sl})^2)}{\textrm{erf}(x^{*}_{sl})}
- \frac{\sqrt{\pi}x^{*}_{sl}L}
{C_l(T_f - T_w)}
= 0
$$

We code the left-hand side (LHS) of Eqn 6.

In [ ]:
def solid_liquid(xx_sl):
    """
    Left-hand side (LHS) of Eqn. 6.
    
    We should find xx_sl such that this function is zero.
    
    *Warning* Do not evaluate this function close to $x^{*}_{sl} = 0$, since it will diverge there.
    """
    global T_f, T_0, T_w
    global k_l, k_s
    global µ_s, µ_l
    global C_s, C_l
    global L
    
    A1 = (T_f - T_0) / (T_f - T_w)
    A2 = k_s/k_l
    A3 = sqrt(µ_l/µ_s)
    A4 = exp(-(µ_l/µ_s)*xx_sl**2) / erfc(sqrt(µ_l/µ_s)*xx_sl)
    A = A1*A2*A3*A4
    
    B = exp(-xx_sl**2) / erf(xx_sl)
    
    C = (sqrt(pi)*xx_sl*L) / (C_l * (T_f - T_w))
    
    return A+B-C

And we code Eqn. 5

In [ ]:
def T_profile_MCM2009(x, t, xxsl):
    global T_w, T_f
    global T_0
    global µ_l
    global µ_s
    xx = x/(2*sqrt(µ_l*t))
    T = np.zeros_like(xx)
    T[xx<xxsl0] = T_w + (T_f-T_w) * (erf(xx[xx<xxsl])/erf(xxsl))
    T[xx>=xxsl0] = T_0 + (T_f-T_0) * (erfc(sqrt(µ_l/µ_s)*xx[xx>=xxsl])/erfc(sqrt(µ_l/µ_s)*xxsl))
    return T

We give realistic values to the physical parameters. **PLEASE CHECK THESE PARAMETERS!**

In [ ]:
T_0 = -10. # initial ice temperature °C
T_w = 20. # external boundary temperature °C
T_f = 0. # melting temperature °C

C_s = 2108. # ice heat capacity J/(K.kg)
C_l = 4184. # water heat capacity J/(K.kg)

µ_s = 1.02e-6  # ice thermal diffusivity
µ_l = 0.143e-6 # water thermal diffusivity (m^2/s)

k_s = 2.2199 # ice thermal conductivity W/(m.K)
k_l = 0.5918 # water thermal conductivity W/(m.K)

L = -334000. # latent heat (J / kg)

With these physical parameters set, Eqn. 6 of MCM needs to be solved (numerically) for $x^{*}_{sl}$.

We have the function, `solid_liquid(xx_sl)`, which evaluates the left-hand side (LHS) of Eqn. 6. Now, we will proceed to find the zero of this LHS function.

$x^{*}_{sl}$ is called `xx_sl` in the code.

With the above parameters, there is indeed an interval over $x^{*}_{sl}$ where the LHS (`solid_liquid(xx_sl)`) is zero, as demonstrated in the following figure.

In [ ]:
xx_sl = np.linspace(0.1,0.6,1000)

In [ ]:
plt.plot(xx_sl, solid_liquid(xx_sl))
plt.xlabel('xx_sl')
plt.ylabel('solid_liquid(xx_sl)')

The zero of the function can be found numerically using the secant method (which is Newton's method, but using numerical approximation of the value of the derivative instead of analytic derivative).

We use our own secant solver (the secant solver in `scipy` contains a bug). It can be found here: https://github.com/mhvwerts/Python-mecatronique/tree/master/TPT_extra

In [ ]:
from secant import secant

Now we use `secant()` to solve for $x^{*}_{sl}$ (= `xx_sl`).

In [ ]:
xxsl0 = secant()

In [ ]:
print(xxsl0)

Here we plot this graphically. The red dot is indeed where LHS is zero.

In [ ]:
plt.plot(xx_sl, solid_liquid(xx_sl),'k')
plt.plot([xx_sl[0],xx_sl[-1]],[0.,0.])
plt.plot([xxsl0],[0.],'ro')
plt.xlabel('xx_sl')
plt.ylabel('solid_liquid(xx_sl)')

Now we look to evaluate $T(x, t)$, the evolution of the temperature profile over time.

Figure 2(a) in MCM2009 gives a temperature profile at 50 hours. Let's try to reproduce that using the expressions given in MCM2009.

In [ ]:
x = np.linspace(0,0.5,1000)
t = 3600.0 * 50. # 50 hours
T = T_profile_MCM2009(x, t, xxsl0)

In [ ]:
plt.plot(x,T)
plt.xlabel('depth / m')
plt.ylabel('temperature / °C')
plt.title('reproduction MCM2009 Figure 2(a)')

This reproduces Figure 2(a)!

Let's see at other time steps. (and include 50h point)

In [ ]:
x = np.linspace(0,0.5,1000)
T = np.zeros_like(x)
for t in [0.1, 1., 10., 100., 1000., 10000., 3600.*50.]:
    T = T_profile_MCM2009(x, t, xxsl0)
    plt.plot(x,T, label='t={0:.1f}'.format(t))
plt.xlabel('depth (m)')
plt.ylabel('temperature (°C)')
plt.legend()

In [ ]:
x = 0.05
t = np.linspace(????,????,500)
T = T_profile_MCM2009(x, t, xxsl0)

In [ ]:
plt.plot(t/3600,T)
plt.xlabel('time (hr)')
plt.ylabel('temperature (°C)')
plt.title('reproduction MCM2009 Figure 2(b)')